In [2]:
# from model.model_type import ModelType
from service.site_scraper import SiteScraper

gsp_scraper = SiteScraper(
    name="gsp",
    base_url="https://www.gsp.ro",
    traffic=3.6,
    time_selector="p.data-author",
    block_selector="div.news-item",
    link_selector="h2 a",
    title_strategy="text"
)

digisport_scraper = SiteScraper(
    name="digisport",
    base_url="https://www.digisport.ro",
    traffic=3.3,
    time_selector="cite",
    block_selector="article.article",
    link_selector="a.article-link, h3 a.widget-latest-list-item-link",
    title_strategy="attribute",
    title_attribute="title",
    # model=ModelType.SPACY
)

fanatik_scraper = SiteScraper(
    name="fanatik",
    base_url="https://www.fanatik.ro",
    traffic=2.9,  # Estimated, adjust as needed
    time_selector="span.date",  # Often used near article blocks
    block_selector="div.article",  # Container for each article
    link_selector="h3 a",  # Headline links
    title_strategy="text",
    # model=ModelType.GPT
)

prosport_scraper = SiteScraper(
    name="prosport",
    base_url="https://www.prosport.ro",
    traffic=3.1,
    time_selector="span.date",
    block_selector="h2.article__title",  # Directly targets the title block
    link_selector="a",  # The anchor inside the h2
    title_strategy="text",
    # model=ModelType.CLAUDE
)

sites = [gsp_scraper, digisport_scraper, fanatik_scraper, prosport_scraper]
# sites = [prosport_scraper]
total_traffic = sum(site.traffic for site in sites)

In [3]:
for site in sites:
    site.compute_weight(total_traffic)
    site.scrape_recent_articles(minutes=60)
    # site.save_to_csv()
    # site.short_print()


In [4]:
for site in sites:
    site.save_to_csv()
    # site.short_print()

In [5]:
from service.story_clusterer import StoryClusterer

clusterer = StoryClusterer(sites, 360, 0.3, 0.2)
clusterer.cluster_stories()

clusterer.print_matched_clusters()

Shared article https://www.prosport.ro/fotbal-intern/superliga/i-au-facut-lista-de-transferuri-lui-gigi-becali-in-direct-acolo-e-nevoie-de-jucatori-fotbalistul-preferat-al-patronului-de-la-fcsb-entuziasmat-exclusiv-20289509 on digisport - prosport skipped https://www.prosport.ro/fotbal-intern/superliga/i-au-facut-lista-de-transferuri-lui-gigi-becali-in-direct-acolo-e-nevoie-de-jucatori-fotbalistul-preferat-al-patronului-de-la-fcsb-entuziasmat-exclusiv-20289509 digisport prosport

🧠 Cluster #1 — Score: 0.519

🧠 Cluster #2 — Score: 0.496

🧠 Cluster #3 — Score: 0.256

🔍 Matched Clusters Across Multiple Sites

🧠 Cluster #1 — Score: 0.519 — Sites: gsp, prosport
------------------------------------------------------------
📰 Mihaela Cambei e campioană mondială! A luat aurul și încă două medalii de argint în Norvegia
🔗 https://www.prosport.ro/alte-sporturi/mihaela-cambei-e-campioana-mondiala-a-luat-aurul-si-inca-doua-medalii-de-argint-in-norvegia-20289305
🔑 Keywords: cont, câștigat medalia de 

In [ ]:

top_stories = clusterer.score_clusters()

for i, story in enumerate(top_stories[:5], 1):
    print(f"\n🏆 Top {i} — Score: {story['score']}")
    for article in story["articles"]:
        print(f"- {article.site}: {article.title}")
